<a href="https://colab.research.google.com/github/mithunboraiah007/project1/blob/main/stock_price_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [135]:

import yfinance as yf
import pandas as pd
import numpy as np

In [136]:
ticker_symbol = input("Enter stock symbol (e.g., GAIL.NS): ")
start_date = input("Enter start date (YYYY-MM-DD): ")
end_date = input("Enter end date (YYYY-MM-DD): ")

Enter stock symbol (e.g., GAIL.NS): GAIL.NS
Enter start date (YYYY-MM-DD): 2024-01-01
Enter end date (YYYY-MM-DD): 2024-05-01


In [137]:
stock_data = yf.download(ticker_symbol, start=start_date, end=end_date)

[*********************100%%**********************]  1 of 1 completed


In [138]:
stock_data

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2024-01-01,163.699997,169.350006,162.649994,166.199997,161.166412,40952729
2024-01-02,167.100006,167.600006,158.100006,164.050003,159.081528,40801901
2024-01-03,164.000000,164.399994,160.750000,162.149994,157.239075,22338899
2024-01-04,162.300003,165.699997,162.250000,163.149994,158.208786,16795469
2024-01-05,166.000000,167.100006,161.899994,163.649994,158.693634,36182162
...,...,...,...,...,...,...
2024-04-24,200.899994,208.800003,199.199997,207.399994,207.399994,31299709
2024-04-25,207.300003,209.000000,206.100006,208.000000,208.000000,12365830
2024-04-26,208.899994,211.699997,207.300003,208.050003,208.050003,13324638


In [139]:
stock_data['Next_Day_Trend'] = np.where(stock_data['Close'].shift(-1) > stock_data['Close'], 1, 0)

In [140]:
clean_stock_data = stock_data.fillna(method='ffill')

<ipython-input-140-b1fbb0e3c815>:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  clean_stock_data = stock_data.fillna(method='ffill')


In [141]:
clean_stock_data['Short_MA'] = clean_stock_data['Close'].rolling(window=10).mean()
clean_stock_data['Long_MA'] = clean_stock_data['Close'].rolling(window=50).mean()

In [142]:
delta = clean_stock_data['Close'].diff()
gain = (delta.where(delta > 0, 0)).rolling(window=50).mean()
loss = (-delta.where(delta < 0, 0)).rolling(window=50).mean()
RS = gain / loss
RSI = 100 - (100 / (1 + RS))
clean_stock_data['RSI'] = RSI

# Drop NaN values generated by rolling calculations
clean_stock_data.dropna(inplace=True)

In [143]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from tensorflow.keras.models import Sequential

In [144]:
clean_stock_data['Next_Day_Trend'] = np.where(clean_stock_data['Close'].shift(-1) > clean_stock_data['Close'], 1, 0)

In [145]:
features = ['Short_MA', 'Long_MA', 'RSI', 'Next_Day_Trend']
target = 'Close'

In [146]:
train_size = int(0.8 * len(clean_stock_data))
train_data = clean_stock_data[:train_size]
test_data = clean_stock_data[train_size:]


In [147]:
# Training using Support Vector Machine (SVR)
model_svr = SVR(kernel='linear')
model_svr.fit(train_data[features], train_data[target])

SVR(kernel='linear')

In [148]:
# Training using Random Forest
model_rf = RandomForestRegressor(n_estimators=100, random_state=42)
model_rf.fit(train_data[features], train_data[target])

RandomForestRegressor(random_state=42)

In [149]:
# Evaluation
def evaluate_model(model, test_features, test_target):
    predictions = model.predict(test_features)
    mse = mean_squared_error(test_target, predictions)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(test_target, predictions)
    r2 = r2_score(test_target, predictions)
    return mse, rmse, mae, r2


In [150]:
# Evaluate SVR model
mse_svr, rmse_svr, mae_svr, r2_svr = evaluate_model(model_svr, test_data[features], test_data[target])
print("Support Vector Machine (SVR):")
print(f"MSE: {mse_svr}, RMSE: {rmse_svr}, MAE: {mae_svr}, R-squared: {r2_svr}")

Support Vector Machine (SVR):
MSE: 42.63748387015782, RMSE: 6.529738422797488, MAE: 4.43138323224522, R-squared: -1.388870524779378


In [151]:

# Evaluate Random Forest model
mse_rf, rmse_rf, mae_rf, r2_rf = evaluate_model(model_rf, test_data[features], test_data[target])
print("Random Forest:")
print(f"MSE: {mse_rf}, RMSE: {rmse_rf}, MAE: {mae_rf}, R-squared: {r2_rf}")

Random Forest:
MSE: 23.902843168279883, RMSE: 4.889053402068736, MAE: 4.714641723632817, R-squared: -0.3392159273988742


In [152]:
model_mse = {
    'SVR': mse_svr,
    'Random Forest': mse_rf,
}

In [154]:
future_days = 1  # Replace with the desired number of future days

last_short_ma = clean_stock_data['Short_MA'].iloc[-1]
last_long_ma = clean_stock_data['Long_MA'].iloc[-1]
last_rsi = clean_stock_data['RSI'].iloc[-1]
last_trend = clean_stock_data['Next_Day_Trend'].iloc[-1]

future_short_ma_values = []  # Store future Short_MA values
future_long_ma_values = []  # Store future Long_MA values
future_rsi_values = []  # Store future RSI values
future_trend_values = []  # Store future Next_Day_Trend values

for i in range(future_days):
    # Example: Suppose you're predicting the next day's trend based on today's values
    # Replace this with your own forecasting logic
    future_short_ma_values.append(last_short_ma + 1)  # Placeholder logic
    future_long_ma_values.append(last_long_ma + 1)  # Placeholder logic
    future_rsi_values.append(last_rsi + 1)  # Placeholder logic
    future_trend_values.append(last_trend + 1)  # Placeholder logic

    # Update the last known values for the next iteration
    last_short_ma = future_short_ma_values[-1]
    last_long_ma = future_long_ma_values[-1]
    last_rsi = future_rsi_values[-1]
    last_trend = future_trend_values[-1]

# Construct a DataFrame for future features
future_features = pd.DataFrame({
    'Short_MA': future_short_ma_values,
    'Long_MA': future_long_ma_values,
    'RSI': future_rsi_values,
    'Next_Day_Trend': future_trend_values
})

# Predict open prices for the next 'n' days using the selected model (e.g., Linear Regression)
predicted_open_prices = model_rf.predict(future_features)
print("Predicted future open prices:")
print(predicted_open_prices)

Predicted future open prices:
[203.15250168]
